<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

# Info

Notatnik stworzony do eksperymentowania z kwantyzacją modeli. Zaimplementowano w nim trzy metody: GPTQ, AWQ i GGUF. Każda z nich pozwala zmniejszyć rozmiar i wymagania obliczeniowe modelu, jednocześnie poprawiając jego wydajność. Notatnik umożliwia badanie wpływu każdej z technik, dając praktyczne doświadczenie z nowoczesną optymalizacją modeli i jest przydatny również dla osób z ograniczonymi zasobami sprzętowymi.

## Instalacja bibliotek

In [ ]:
%%capture

!pip install accelerate -q
!pip install optimum -q
!pip install auto-gptq -q
!pip install ctransformers[cuda]

In [ ]:
import gc
import torch
import warnings

warnings.filterwarnings("ignore") # zastosowane w celach demonstracyjnych

## **GPTQ**
Gradient Projection for Quantization (GPTQ)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "speakleash/Bielik-7B-Instruct-v0.1-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')
gptq=True

In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]  1. Kazimierz Dolny: Miasteczko położone nad Wisłą, słynące z malowniczych ruin zamku, wąwozów lessowych i uroczego rynku. Znane z letnich festiwali i zimowych imprez.

2. Gdańsk: Historyczne miasto z portem, położone nad Morzem Bałtyckim. Znane z zabytkowej architektury, Starego Miasta i Bazyliki Mariackiej.

3. Wrocław: Stolica Dolnego Śląska, znana z zabytkowej architektury, Mostu Grunwaldzkiego i wielu parków. Wrocław jest również znany z festiwali kulturalnych i Mostu Zakochanych.


## **GGUF**
Group-wise Gradient Update Filtering (GGUF)

In [ ]:
# poniższy kod wykona sie jedynie w przypadku, gdy tokenizer został uprzednio 'zainicjalizowany'
if 'model' in globals() and 'tokenizer' in globals() and 'pipe' in globals():
  print("Remove from memory: model, tokenizer and pipe")
  del model, tokenizer, pipe
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "speakleash/Bielik-7B-Instruct-v0.1-GGUF",
    model_file="bielik-7b-instruct-v0.1.Q4_K_M.gguf", # różne typy kwantyzacji można pobrać z repozytorium speakleash/Bielik-7B-Instruct-v0.1-GGUF
    model_type="mistral", gpu_layers=50, hf=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "speakleash/Bielik-7B-Instruct-v0.1", use_fast=True
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
outputs = pipe(prompt, max_new_tokens=256)
print(outputs[0]["generated_text"])

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]  Trzy ciekawe miejscowości w Polsce to:

1. Kraków - dawna stolica Polski, znana z zabytkowej architektury, w tym Wawelu, Sukiennic i Kościoła Mariackiego. Kraków jest również centrum kultury, oferującym festiwale, muzea i teatry.

2. Gdańsk - nadmorskie miasto z długą historią. Znane z Bazyliki Mariackiej, Dworu Artusa i fontanny Neptuna. Gdańsk jest również ważnym portem morskim i centrum biznesu.

3. Wrocław - miasto położone na Odrze, znane z licznych mostów i zabytkowej architektury. Wrocław jest Europejską Stolicą Kultury i słynie z festiwali, takich jak Wrocławski Festiwal Filmowy. Miasto jest również domem dla bogatej sieci park


## **AWQ**
Adaptive Weight Quantization (AWQ)

**UWAGA**: Aby użyć poniższego przykładu `AWQ`, należy uprzednio odłączyć środowisko wykonawcze przed zainstalowaniem zależności `vllm`. Mogą wystąpić konflikty zależności z pakietami, które zainstalowaliśmy wcześniej, więc najlepiej zacząć od zera.

In [ ]:
# restart środowiska Colab
exit()

In [ ]:
%%capture

!pip install vllm

In [ ]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=256)
llm = LLM(
    model="speakleash/Bielik-7B-Instruct-v0.1-AWQ",
    quantization='awq',
    dtype='half',
    gpu_memory_utilization=.95,
    max_model_len=4096
)

In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
output = llm.generate(prompt, sampling_params)
print(output[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.84s/it]

  Trzy ciekawe miejscowości w Polsce to:
1. Gdańsk - nadmorskie miasto z bogatą historią i kulturą, znane z Bazyliki Mariackiej, Dworu Artusa, fontanny Neptuna i Muzeum II Wojny Światowej.
2. Kraków - historyczne miasto z piękną starówką, Wawelskim Zamkiem, Sukiennicami i Kościołem Mariackim.
3. Wrocław - miasto z malowniczymi kanałami, Starym Rynkiem i zabytkami takimi jak Ostrów Tumski, Most Pokoju i Halą Stulecia.
